In [1]:
import requests
from pprint import pprint
import re,json

In [4]:
#Function to check the API connection 
def fetch_data_from_api(api_url):
    try:
        response = requests.get(api_url)
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            data = response.json()  # Assuming the API returns JSON data
            return data
        else:
            print("Failed to fetch data from API. Status code:", response.status_code)
            return None
    except Exception as e:
        print("An error occurred:", e)
        return None

In [60]:
# Example usage:
api_url = "https://api.mfapi.in/mf"  # Replace this with the actual API endpoint
data = fetch_data_from_api(api_url)

if data:
    # Do something with the fetched data
    print("Data Captured")
else:
    print("Failed to fetch data from the API.")

Data Captured


In [61]:
required_keywords = [["Direct","Growth","ELSS"], ["direct", "growth","ELSS"],["Direct","Growth","TAX"]]
non_req_keywords = [["Bonus"]]

In [77]:
tax_saving_lst =[]
for fund in data:
    for substrings in required_keywords:
        if all(substring in fund['schemeName'] for substring in substrings):
            for nonstrings in non_req_keywords:
                if all(string not in fund['schemeName'] for string in nonstrings):
                    tax_saving_lst.append(fund)

In [78]:
pprint(tax_saving_lst)

[{'schemeCode': 111549,
  'schemeName': 'Quantum ELSS Tax Saver Fund - Direct Plan Growth Option'},
 {'schemeCode': 118473,
  'schemeName': 'BANDHAN ELSS Tax Saver Fund - Direct Plan - Growth'},
 {'schemeCode': 118473,
  'schemeName': 'IDFC Tax Advantage  (ELSS) Fund-Direct Plan-Growth'},
 {'schemeCode': 118473,
  'schemeName': 'BANDHAN Tax Advantage (ELSS) Fund-Direct Plan-Growth'},
 {'schemeCode': 118473,
  'schemeName': 'IDFC Tax Advantage  (ELSS) Fund-Direct Plan-Growth '},
 {'schemeCode': 118540,
  'schemeName': 'Franklin India TAXSHIELD - Direct - Growth'},
 {'schemeCode': 118540,
  'schemeName': 'Franklin India ELSS Tax Saver Fund - Direct - Growth'},
 {'schemeCode': 118620,
  'schemeName': 'Edelweiss ELSS Tax Saver Fund - Direct Plan-Growth Option'},
 {'schemeCode': 118866,
  'schemeName': 'Taurus ELSS Tax Saver Fund - Direct Plan - Growth'},
 {'schemeCode': 119060,
  'schemeName': 'HDFC ELSS Tax saver - Growth Option - Direct Plan'},
 {'schemeCode': 119242,
  'schemeName': 'DS

In [79]:
file_name = "output/mf_elss.json"

# Writing the JSON list to a file
with open(file_name, 'w') as json_file:
    json.dump(tax_saving_lst, json_file)

print("JSON file saved successfully.")

JSON file saved successfully.


In [4]:
import os
import json
import requests

class SchemeFilter:
    def __init__(self, api_url):
        self.api_url = api_url
    
    def fetch_api_data(self):
        response = requests.get(self.api_url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data from API. Status code: {response.status_code}")
            return None
    
    def filter_by_substring(self, filename, include_substrings=None, exclude_substrings=None):
        api_data = self.fetch_api_data()
        if api_data is None:
            return
        
        if include_substrings is None:
            include_substrings = []
        if exclude_substrings is None:
            exclude_substrings = []
        
        filtered_data = []
        for scheme in api_data:
            scheme_name = scheme["schemeName"]
            include_matched = any(all(substring in scheme_name for substring in sublist) for sublist in include_substrings)
            exclude_matched = all(substring not in scheme_name for substring in exclude_substrings)
            
            if include_matched and exclude_matched:
                filtered_data.append(scheme)
        
        output_dir = "output"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        output_path = os.path.join(output_dir, filename)
        with open(output_path, "w") as outfile:
            json.dump(filtered_data, outfile)

In [3]:
# Example usage:
api_url = "https://api.mfapi.in/mf"
filter_instance = SchemeFilter(api_url)
include_lists = [["Growth", "Direct","TAX"], ["Growth", "Direct","ELSS"]]
exclude_list = ["Dividend"]
output_filename = "elss_mf.json"
filter_instance.filter_by_substring(output_filename, include_substrings=include_lists, exclude_substrings=exclude_list)